In [5]:
!pip install requests tqdm

import os
os.environ["ODCLOUD_SERVICE_KEY"] = os.environ.get("ODCLOUD_SERVICE_KEY","").strip() or "ZJNBMKBH6C2i9ljEZjHs/UC6HmjKFxRal8APQh5bO3Wf2r/fkQTB8fhI27Qt+xlTPkt+mG1nIBluxnR3xZQT4Q=="



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
# === [셀 2] 상수 및 임포트 ===
import os, re, json, time, html, sqlite3, textwrap
from typing import Any, Dict, Iterable, List, Optional, Tuple
from datetime import datetime
import requests
from tqdm import tqdm

SERVICE_KEY = os.getenv("ODCLOUD_SERVICE_KEY", "").strip()

# 엔드포인트 후보(가끔 서브도메인 바뀜 대비)
BASE_CANDIDATES = [
    "https://api.odcloud.kr/api/gov24/v3",
    "https://infuser.odcloud.kr/api/gov24/v3",
]
ENDPOINTS = {
    "list": "serviceList",
    "detail": "serviceDetail",
    "cond": "serviceCondition",
}
DEFAULT_PER_PAGE = 100
UA = "gov24-collector/0.2 (+https://example.local)"  # 아무 UA나
TIMEOUT = 15

# 코드→사람친화 라벨(참고용)
CODE_TO_LABEL: Dict[str, str] = {
    "서비스ID":"서비스ID","서비스명":"서비스명",
    "JA0101":"남성","JA0102":"여성","JA0110":"대상연령(시작, 만)","JA0111":"대상연령(종료, 만)",
    "JA0201":"중위소득 0~50%","JA0202":"중위소득 51~75%","JA0203":"중위소득 76~100%","JA0204":"중위소득 101~200%","JA0205":"중위소득 200% 초과",
    "JA0301":"예비부모/난임","JA0302":"임산부","JA0303":"출산/입양","JA0313":"농업인","JA0314":"어업인","JA0315":"축산업인","JA0316":"임업인",
    "JA0317":"초등학생","JA0318":"중학생","JA0319":"고등학생","JA0320":"대학생/대학원생","JA0322":"개인대상 해당없음",
    "JA0326":"근로자/직장인","JA0327":"구직자/실업자","JA0328":"장애인","JA0329":"국가보훈대상자","JA0330":"질병/질환자",
    "JA0401":"다문화가족","JA0402":"북한이탈주민","JA0403":"한부모/조손가정","JA0404":"1인가구","JA0410":"가구특성 해당없음",
    "JA0411":"다자녀가구","JA0412":"무주택세대","JA0413":"신규전입","JA0414":"확대가족",
    "JA1101":"예비창업자","JA1102":"영업중","JA1103":"생계곤란/폐업예정",
    "JA1201":"음식점업","JA1202":"제조업","JA1299":"기타업종(개인)",
    "JA2101":"중소기업","JA2102":"사회복지시설","JA2103":"기관/단체",
    "JA2201":"제조업(기업/기관)","JA2202":"농임어업(기업/기관)","JA2203":"정보통신업(기업/기관)","JA2299":"기타업종(기업/기관)",
}


In [7]:
# === [셀 3] 텍스트 정규화 / 날짜·금액 파싱 / 키워드 매칭 / 조건 추출 (최종 통합본) ===
import re, html, calendar
from datetime import datetime
from typing import Any, Dict, List, Optional, Tuple

# 1) 텍스트 정리
def clean_text(x: Any) -> Any:
    if x is None:
        return None
    if not isinstance(x, str):
        return x
    x = html.unescape(x)
    x = re.sub(r"\s+", " ", x).strip()
    return x or None

# 2) 날짜 파싱 (보강)
def normalize_date_ymd(y: int, m: int, d: int) -> Optional[str]:
    try:
        return f"{y:04d}-{m:02d}-{d:02d}"
    except:
        return None

def last_day_of_month(y: int, m: int) -> int:
    return calendar.monthrange(y, m)[1]

def normalize_date(s: Optional[str]) -> Optional[str]:
    if not s:
        return None
    s = clean_text(s)
    # YYYY.MM.DD (한글 섞여도 허용)
    m = re.search(r"(\d{4})[.\-/년\s]*(\d{1,2})[.\-/월\s]*(\d{1,2})", s)
    if m:
        return normalize_date_ymd(int(m.group(1)), int(m.group(2)), int(m.group(3)))
    # YYYY.MM (일자 없음 → 1일로 보정)
    m = re.search(r"(\d{4})[.\-/년\s]*(\d{1,2})", s)
    if m:
        return normalize_date_ymd(int(m.group(1)), int(m.group(2)), 1)
    return None

def parse_dates_from_text(text: Optional[str]) -> Tuple[Optional[str], Optional[str]]:
    """자유서술식 기간에서 (from, to) 추출. 없으면 (None, None)."""
    if not text:
        return (None, None)
    t = clean_text(text)

    # 상시/연중/수시/예산소진/별도 공지
    if re.search(r"(상시|연중|수시|예산\s*소진\s*시\s*까지|예산소진시까지|별도\s*공지)", t):
        return (None, None)

    # 1) YYYY.MM.DD ~ YYYY.MM.DD
    m = re.search(r"(\d{4}[.\-/]\d{1,2}[.\-/]\d{1,2})\s*[~\-–]\s*(\d{4}[.\-/]\d{1,2}[.\-/]\d{1,2})", t)
    if m:
        return (normalize_date(m.group(1)), normalize_date(m.group(2)))

    # 2) YYYY.MM.DD ~ MM.DD  (뒤 연도 생략 → 앞 연도 재사용)
    m = re.search(r"(\d{4}[.\-/]\d{1,2}[.\-/]\d{1,2})\s*[~\-–]\s*(\d{1,2}[.\-/]\d{1,2})", t)
    if m:
        d1 = normalize_date(m.group(1))
        y1 = int(d1[:4]) if d1 else datetime.utcnow().year
        mmdd = re.search(r"(\d{1,2})[.\-/](\d{1,2})", m.group(2))
        d2 = normalize_date_ymd(y1, int(mmdd.group(1)), int(mmdd.group(2)))
        return (d1, d2)

    # 3) YYYY.MM ~ YYYY.MM  (월만 → 1일/말일로 보정)
    m = re.search(r"(\d{4}[.\-/]\d{1,2})\s*[~\-–]\s*(\d{4}[.\-/]\d{1,2})", t)
    if m:
        y1, m1 = map(int, re.findall(r"\d+", m.group(1))[:2])
        y2, m2 = map(int, re.findall(r"\d+", m.group(2))[:2])
        d1 = normalize_date_ymd(y1, m1, 1)
        d2 = normalize_date_ymd(y2, m2, last_day_of_month(y2, m2))
        return (d1, d2)

    # 4) 단일 마감: '~ YYYY.MM.DD' / 'YYYY.MM.DD까지'
    m = re.search(r"(?:까지|~)\s*(\d{4}[.\-/]\d{1,2}[.\-/]\d{1,2})", t)
    if m:
        d = normalize_date(m.group(1))
        return (None, d)

    # 5) 단일 날짜 하나만 있음
    m = re.search(r"(\d{4}[.\-/]\d{1,2}[.\-/]\d{1,2})", t)
    if m:
        d = normalize_date(m.group(1))
        return (d, d)

    return (None, None)

# 3) 금액 파싱 (보강)
_AMT2 = re.compile(
    r"(?:(?:월|회|1회|매월)\s*)?"              # 단가 접두어 허용(캡처X)
    r"([0-9]{1,3}(?:,[0-9]{3})*|[0-9]+)\s*"     # 숫자
    r"(?:(천|만|억)\s*)?"                       # 단위(선택)
    r"원"                                       # 원
)

def parse_amounts(text: Optional[str]) -> Tuple[Optional[int], Optional[int]]:
    if not text:
        return (None, None)
    t = clean_text(text)
    vals: List[int] = []
    for m in _AMT2.finditer(t):
        n = int(m.group(1).replace(",", ""))
        unit = m.group(2)
        mul = 1
        if unit == "천": mul = 1_000
        elif unit == "만": mul = 10_000
        elif unit == "억": mul = 100_000_000
        vals.append(n * mul)
    if not vals:
        return (None, None)
    return (min(vals), max(vals))

# 4) 로컬 키워드 매칭(백업 검색용)
def matches_keyword(row: Dict[str, Any], keyword: str) -> bool:
    keyword = keyword.strip().lower()
    fields = ["서비스명", "서비스목적요약", "지원대상", "지원내용", "서비스분야", "소관기관명"]
    for f in fields:
        v = clean_text(row.get(f))
        if v and keyword in v.lower():
            return True
    return False

# 5) 조건 자동 추출 및 분류 (가장 성능 좋은 로직으로 교체)
COND_KEY_PATTERN = re.compile(
    r"(지원조건|신청자격|지원대상|대상자|선정기준|자격요건|제외대상)"
    r"|(\\b연령|\\b나이|연령기준|소득|중위소득|재산|거주|지역|국적|가구|학력|재학|졸업|고용|취업|실업)",
    re.IGNORECASE
)
# 값 안에서 조건을 암시하는 어휘
VALUE_HINT = re.compile(
    r"(신청자격|지원대상|선정기준|자격요건|제외대상|조건)"
    r"|(\\b연령|\\b나이|중위소득|소득|재산|거주|지역|국적|가구|학력|재학|졸업|고용|취업|실업)",
    re.IGNORECASE
)

def extract_conditions_from_any(obj: Any, bucket: Dict[str, List[str]]):
    """
    dict/list/str 어디에 숨어 있어도 문자열을 끄집어내고,
    (1) 키 이름에 조건 패턴이 보이거나 (2) 값 문자열에 조건 힌트가 보이면
    적절한 카테고리에 담는다.
    """
    def push(cat: str, txt: str):
        txt = clean_text(txt)
        if not txt: return
        arr = bucket.setdefault(cat, [])
        if txt not in arr:
            arr.append(txt)

    if isinstance(obj, dict):
        for k, v in obj.items():
            k_str = str(k)
            if isinstance(v, str):
                # 키 기반
                if COND_KEY_PATTERN.search(k_str):
                    cat = ("age" if any(w in k_str for w in ["연령","나이"])
                           else "income" if "소득" in k_str
                           else "residence" if any(w in k_str for w in ["거주","지역"])
                           else "target" if any(w in k_str for w in ["지원대상","대상자"])
                           else "selection" if "선정기준" in k_str
                           else "eligibility")
                    push(cat, v)
                # 값 기반 (키가 일치하지 않을 경우)
                else:
                    if VALUE_HINT.search(v):
                        low = v.lower()
                        if any(w in low for w in ["연령","나이","age", "세"]): push("age", v)
                        if any(w in low for w in ["중위소득","소득","income"]): push("income", v)
                        if any(w in low for w in ["거주","지역","residence"]): push("residence", v)
                        if any(w in low for w in ["지원대상","대상자","target"]): push("target", v)
                        if any(w in low for w in ["선정기준","selection"]): push("selection", v)
                        if any(w in low for w in ["신청자격","자격요건","조건","eligibility"]): push("eligibility", v)
            # 하위 구조 재귀
            else:
                extract_conditions_from_any(v, bucket)
    elif isinstance(obj, list):
        for x in obj:
            extract_conditions_from_any(x, bucket)
    elif isinstance(obj, str):
        if VALUE_HINT.search(obj):
            low = obj.lower()
            if any(w in low for w in ["연령","나이","age", "세"]): push("age", obj)
            if any(w in low for w in ["중위소득","소득","income"]): push("income", obj)
            if any(w in low for w in ["거주","지역","residence"]): push("residence", obj)
            if any(w in low for w in ["지원대상","대상자","target"]): push("target", obj)
            if any(w in low for w in ["선정기준","selection"]): push("selection", obj)
            if any(w in low for w in ["신청자격","자격요건","조건","eligibility"]): push("eligibility", obj)

def merge_conditions(*cond_maps: Dict[str, List[str]]) -> Dict[str, str]:
    """여러 곳에서 추출된 조건 딕셔너리들을 하나로 병합합니다."""
    merged: Dict[str, List[str]] = {}
    for m in cond_maps:
        for k, vs in (m or {}).items():
            for v in vs:
                merged.setdefault(k, [])
                if v not in merged[k]:
                    merged[k].append(v)
    # 각 카테고리별 텍스트를 줄바꿈 문자로 합쳐서 반환합니다.
    return {k: "\n".join(vs) for k, vs in merged.items()}

def extract_refined_conditions(text: Optional[str]) -> Dict[str, str]:
    """
    텍스트를 문장(줄) 단위로 분해하고, 각 문장이 특정 조건(나이, 소득)에
    해당하는지 개별적으로 검사하여 정확도를 극대화합니다.
    """
    if not text:
        return {}

    # 정규표현식 패턴 정의
    # (?:...) -> 그룹으로 묶지만 캡처는 하지 않음
    # \s* -> 공백 0개 이상
    # [\d,]+ -> 숫자 또는 쉼표 1개 이상
    age_regex = re.compile(r'(?:만\s*)?[\d,]+\s*세(?:\s*이상|이하|미만|부터)?(?:\s*~\s*(?:만\s*)?[\d,]+\s*세)?')
    income_regex = re.compile(r'(?:소득|급여|재산|자산)\s*.*?\s*[\d,]+\s*(?:만\s*원|%)\s*(?:이하|이상|미만|이내|초과)?')

    refined: Dict[str, List[str]] = {'age': [], 'income': []}

    # 텍스트를 줄바꿈 기준으로 분해하여 한 줄씩 검사
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue

        # 각 줄에 대해 나이 패턴과 소득 패턴을 독립적으로 검사
        age_matches = age_regex.findall(line)
        income_matches = income_regex.findall(line)

        # "소득"과 "나이"가 한 문장에 같이 있으면 각각 추가
        if age_matches:
            refined['age'].extend(age_matches)
        if income_matches:
            refined['income'].extend(income_matches)

    # 중복 제거 및 최종 결과 정리
    final_refined = {}
    if refined['age']:
        final_refined['age'] = "\n".join(sorted(list(set(refined['age']))))
    if refined['income']:
        final_refined['income'] = "\n".join(sorted(list(set(refined['income']))))

    return final_refined


In [8]:
# === [셀 4] HTTP 헬퍼/베이스 감지/페이징 수집 ===

def safe_get(session: requests.Session, url: str, params: Dict[str, Any]) -> requests.Response:
    headers = {"User-Agent": UA}
    r = session.get(url, params=params, headers=headers, timeout=TIMEOUT)
    # odcloud는 200이라도 payload code로 실패 알려주는 경우가 있음
    if not r.ok:
        raise requests.HTTPError(f"HTTP {r.status_code} for {r.url}", response=r)
    return r

def jget(session: requests.Session, url: str, params: Dict[str, Any]) -> Dict[str, Any]:
    r = safe_get(session, url, params)
    try:
        data = r.json()
    except Exception as e:
        raise RuntimeError(f"JSON decode 실패: {r.text[:200]}") from e
    # payload 표준화
    if isinstance(data, dict) and "code" in data and data.get("code") not in (0, "0"):
        # odcloud 포맷: {"code": -3, "msg": "..."}
        # 하지만 정상에서도 {"code":0,"msg":"정상"}만 있고 data는 다른 키에 있음
        pass
    return data

def detect_base(session: Optional[requests.Session] = None) -> str:
    session = session or requests.Session()
    for base in BASE_CANDIDATES:
        url = f"{base}/{ENDPOINTS['list']}"
        try:
            probe = jget(session, url, {"page": 1, "perPage": 1, "serviceKey": SERVICE_KEY})
            # 정상: {"code":0,"msg":"정상","data":[...],"currentCount":1,...}
            if isinstance(probe, dict) and ("data" in probe or "currentCount" in probe or "totalCount" in probe):
                return base
        except Exception:
            continue
    # 마지막이라도 api.odcloud로
    return BASE_CANDIDATES[0]

def fetch_paged_unfiltered(
    session: requests.Session,
    url: str,
    base_params: Dict[str, Any],
    per_page: int = DEFAULT_PER_PAGE,
    hard_limit_rows: Optional[int] = None,
    max_pages: Optional[int] = None,
) -> Iterable[Dict[str, Any]]:
    page = 1
    yielded = 0
    while True:
        params = dict(base_params)
        params.update({"page": page, "perPage": per_page})
        data = jget(session, url, params)
        rows = (data.get("data") or [])
        for r in rows:
            yield r
            yielded += 1
            if hard_limit_rows and yielded >= hard_limit_rows:
                return
        current = data.get("currentCount") or len(rows)
        total = data.get("totalCount")
        # 종료조건
        if (current is None or current < per_page) and (total is None or yielded >= total):
            break
        page += 1
        if max_pages and page > max_pages:
            break
        time.sleep(0.15)  # 과도 호출 방지

In [9]:
# === [셀 5] 상세/조건 API 호출 ===

def fetch_detail(session: requests.Session, base: str, service_id: str) -> Optional[Dict[str, Any]]:
    url = f"{base}/{ENDPOINTS['detail']}"
    params = {"serviceKey": SERVICE_KEY, "cond[serviceId]": service_id}
    try:
        data = jget(session, url, params)
        # 대부분 {"data":[{...}], "currentCount":1}
        rows = data.get("data") or []
        return (rows[0] if rows else None)
    except Exception:
        return None

def fetch_conditions(session: requests.Session, base: str, service_id: str) -> Optional[Dict[str, Any]]:
    url = f"{base}/{ENDPOINTS['cond']}"
    params = {"serviceKey": SERVICE_KEY, "cond[serviceId]": service_id}
    try:
        data = jget(session, url, params)
        rows = data.get("data") or []
        return (rows[0] if rows else None)
    except Exception:
        return None


In [10]:
# === [셀 6] 어댑터: iter_service_list / fetch_service_detail / fetch_service_cond ===

_session: Optional[requests.Session] = None
_base: Optional[str] = None

def _get_session() -> requests.Session:
    global _session
    if _session is None:
        _session = requests.Session()
    return _session

def _get_base() -> str:
    global _base
    if _base is None:
        _base = detect_base(_get_session())
    return _base

def _list_endpoint_url() -> str:
    return f"{_get_base()}/{ENDPOINTS['list']}"

def _try_remote_keyword_fetch(session, keyword, per_page, max_pages) -> Optional[List[Dict[str, Any]]]:
    tried_params = [
        {"serviceKey": SERVICE_KEY, "cond[srchWord]": keyword},
        {"serviceKey": SERVICE_KEY, "cond[searchTerm]": keyword},
        {"serviceKey": SERVICE_KEY, "srchWord": keyword},
        {"serviceKey": SERVICE_KEY, "searchTerm": keyword},
    ]
    url = _list_endpoint_url()
    rows: List[Dict[str, Any]] = []
    for base_params in tried_params:
        got_any = False
        for r in fetch_paged_unfiltered(
            session, url, base_params, per_page=per_page, hard_limit_rows=None, max_pages=max_pages
        ):
            rows.append(r); got_any = True
        if got_any:
            return rows
    return None  # 원격 키워드 실패 → 로컬 필터로

def iter_service_list(keyword: Optional[str] = None, per_page: int = DEFAULT_PER_PAGE, warm_pages: int = 3, limit: Optional[int] = None):
    session = _get_session()
    url = _list_endpoint_url()
    yielded = 0
    if keyword:
        rows = _try_remote_keyword_fetch(session, keyword, per_page, max_pages=warm_pages)
        if rows is None:
            base_params = {"serviceKey": SERVICE_KEY}
            for r in fetch_paged_unfiltered(session, url, base_params, per_page, hard_limit_rows=None, max_pages=warm_pages):
                if matches_keyword(r, keyword):
                    yield r; yielded += 1
                    if limit and yielded >= limit: return
        else:
            for r in rows:
                if matches_keyword(r, keyword):
                    yield r; yielded += 1
                    if limit and yielded >= limit: return
    else:
        base_params = {"serviceKey": SERVICE_KEY}
        for r in fetch_paged_unfiltered(session, url, base_params, per_page, hard_limit_rows=limit, max_pages=None):
            yield r

def fetch_service_detail(service_id: str) -> Optional[Dict[str, Any]]:
    return fetch_detail(_get_session(), _get_base(), service_id)

def fetch_service_cond(service_id: str) -> Optional[Dict[str, Any]]:
    return fetch_conditions(_get_session(), _get_base(), service_id)


In [11]:
# === [셀 7] SQLite 스키마/FTS/업서트 (최종 수정본) ===

# === [보조 함수] SQLite 연결 열기 ===
import sqlite3
import json

def _open_conn(db_path: str) -> sqlite3.Connection:
    """
    WAL 모드 + busy_timeout + 기본 튜닝을 적용해서 연결 반환
    """
    con = sqlite3.connect(db_path, timeout=60, isolation_level=None)  # autocommit 모드
    cur = con.cursor()
    cur.execute("PRAGMA journal_mode=WAL;")
    cur.execute("PRAGMA synchronous=NORMAL;")
    cur.execute("PRAGMA temp_store=MEMORY;")
    cur.execute("PRAGMA cache_size=-20000;")   # 약 20MB 메모리 캐시
    cur.execute("PRAGMA busy_timeout=8000;")   # 8초
    return con


DDL_MAIN = """
CREATE TABLE IF NOT EXISTS services (
  service_id      TEXT PRIMARY KEY,
  title           TEXT,
  purpose_summary TEXT,
  category        TEXT,
  org_name        TEXT,
  org_type        TEXT,
  org_code        TEXT,
  applicant_type  TEXT,
  support_type    TEXT,
  support_desc    TEXT,
  target_desc     TEXT,
  apply_method    TEXT,
  apply_period    TEXT,
  apply_from      TEXT,
  apply_to        TEXT,
  phone           TEXT,
  detail_url      TEXT,
  created_at      TEXT,
  updated_at      TEXT,
  views           INTEGER,
  min_amount      INTEGER,
  max_amount      INTEGER,
  raw             TEXT,

  /* ▼ 분류된 조건 저장을 위한 컬럼들 ▼ */
  cond_target     TEXT,
  cond_selection  TEXT,
  cond_age        TEXT,
  cond_income     TEXT,
  cond_residence  TEXT,
  cond_eligibility TEXT
);
"""

DDL_FTS = """
CREATE VIRTUAL TABLE IF NOT EXISTS services_fts USING fts5(
  title, purpose_summary, support_desc, target_desc, category, org_name, content='services', content_rowid='rowid'
);
"""

def init_db(db_path: str):
    con = sqlite3.connect(db_path)
    cur = con.cursor()
    cur.execute(DDL_MAIN)
    cur.execute(DDL_FTS)
    con.commit()
    con.close()

def _row_from_item(item: Dict[str, Any], detail: Optional[Dict[str, Any]], cond: Optional[Dict[str, Any]]) -> Dict[str, Any]:
    # --- 1. 기본 정보 추출 ---
    title = clean_text(item.get("서비스명"))
    purpose = clean_text(item.get("서비스목적요약"))
    category = clean_text(item.get("서비스분야"))
    org_name = clean_text(item.get("소관기관명"))
    org_type = clean_text(item.get("소관기관유형"))
    org_code = clean_text(item.get("소관기관코드"))
    applicant_type = clean_text(item.get("사용자구분"))
    support_type = clean_text(item.get("지원유형"))
    support_desc = clean_text(item.get("지원내용") or (detail or {}).get("지원내용"))
    target_desc = clean_text(item.get("지원대상") or (detail or {}).get("지원대상"))
    apply_method = clean_text(item.get("신청방법") or (detail or {}).get("신청방법"))
    phone = clean_text(item.get("전화문의") or (detail or {}).get("전화문의"))
    detail_url = clean_text(item.get("상세조회URL"))
    period_str = clean_text(item.get("신청기한") or (detail or {}).get("신청기한"))
    d_from, d_to = parse_dates_from_text(period_str)
    mn, mx = parse_amounts(support_desc or "")
    created_at = clean_text(item.get("등록일시"))
    updated_at = clean_text(item.get("수정일시"))
    try:
        views = int(item.get("조회수")) if item.get("조회수") is not None else None
    except:
        views = None

    # --- 2. [1단계] 넓은 범위의 조건 텍스트 추출 ---
    a_bucket: Dict[str, List[str]] = {}
    extract_conditions_from_any(item, a_bucket)
    b_bucket: Dict[str, List[str]] = {}
    extract_conditions_from_any(detail, b_bucket)
    c_bucket: Dict[str, List[str]] = {}
    extract_conditions_from_any(cond, c_bucket)
    broad_conds = merge_conditions(a_bucket, b_bucket, c_bucket)

    # --- 3. [2단계] 정밀 조건 추출 ---
    text_to_scan = "\n".join(filter(None, broad_conds.values()))
    refined_conds = extract_refined_conditions(text_to_scan)

    # --- 4. 최종 데이터 조합 ---
    return {
        "service_id": clean_text(item.get("서비스ID")),
        "title": title,
        "purpose_summary": purpose,
        "category": category,
        "org_name": org_name,
        "org_type": org_type,
        "org_code": org_code,
        "applicant_type": applicant_type,
        "support_type": support_type,
        "support_desc": support_desc,
        "target_desc": target_desc,
        "apply_method": apply_method,
        "apply_period": period_str,
        "apply_from": d_from,
        "apply_to": d_to,
        "phone": phone,
        "detail_url": detail_url,
        "created_at": created_at,
        "updated_at": updated_at,
        "views": views,
        "min_amount": mn,
        "max_amount": mx,
        "raw": json.dumps({"list": item, "detail": detail, "cond": cond}, ensure_ascii=False),

        "cond_target": broad_conds.get("target"),
        "cond_selection": broad_conds.get("selection"),
        "cond_eligibility": broad_conds.get("eligibility"),
        "cond_residence": broad_conds.get("residence"),

        "cond_age": refined_conds.get("age"),
        "cond_income": refined_conds.get("income"),
    }

UPSERT_SQL = """
INSERT INTO services(
  service_id, title, purpose_summary, category, org_name, org_type, org_code, applicant_type,
  support_type, support_desc, target_desc, apply_method, apply_period, apply_from, apply_to,
  phone, detail_url, created_at, updated_at, views, min_amount, max_amount, raw,
  cond_target, cond_selection, cond_age, cond_income, cond_residence, cond_eligibility
) VALUES (
  :service_id, :title, :purpose_summary, :category, :org_name, :org_type, :org_code, :applicant_type,
  :support_type, :support_desc, :target_desc, :apply_method, :apply_period, :apply_from, :apply_to,
  :phone, :detail_url, :created_at, :updated_at, :views, :min_amount, :max_amount, :raw,
  :cond_target, :cond_selection, :cond_age, :cond_income, :cond_residence, :cond_eligibility
) ON CONFLICT(service_id) DO UPDATE SET
  title=excluded.title,
  purpose_summary=excluded.purpose_summary,
  category=excluded.category,
  org_name=excluded.org_name,
  org_type=excluded.org_type,
  org_code=excluded.org_code,
  applicant_type=excluded.applicant_type,
  support_type=excluded.support_type,
  support_desc=excluded.support_desc,
  target_desc=excluded.target_desc,
  apply_method=excluded.apply_method,
  apply_period=excluded.apply_period,
  apply_from=excluded.apply_from,
  apply_to=excluded.apply_to,
  phone=excluded.phone,
  detail_url=excluded.detail_url,
  created_at=excluded.created_at,
  updated_at=excluded.updated_at,
  views=excluded.views,
  min_amount=excluded.min_amount,
  max_amount=excluded.max_amount,
  raw=excluded.raw,
  cond_target=excluded.cond_target,
  cond_selection=excluded.cond_selection,
  cond_age=excluded.cond_age,
  cond_income=excluded.cond_income,
  cond_residence=excluded.cond_residence,
  cond_eligibility=excluded.cond_eligibility
;
"""

def upsert_rows(db_path: str, rows: List[Dict[str, Any]]):
    max_tries = 8
    delay = 0.25

    for attempt in range(1, max_tries+1):
        con = None
        try:
            con = _open_conn(db_path)
            cur = con.cursor()

            cur.execute("BEGIN IMMEDIATE;")
            cur.executemany(UPSERT_SQL, rows)

            try:
                cur.execute("DELETE FROM services_fts;")
                cur.execute("""
                    INSERT INTO services_fts(rowid, title, purpose_summary, support_desc, target_desc, category, org_name)
                    SELECT rowid, title, purpose_summary, support_desc, target_desc, category, org_name FROM services;
                """)
            except sqlite3.DatabaseError:
                cur.execute("DROP TABLE IF EXISTS services_fts;")
                cur.execute(DDL_FTS)
                cur.execute("""
                    INSERT INTO services_fts(rowid, title, purpose_summary, support_desc, target_desc, category, org_name)
                    SELECT rowid, title, purpose_summary, support_desc, target_desc, category, org_name FROM services;
                """)

            con.commit()

            try:
                chk = cur.execute("PRAGMA quick_check;").fetchone()
                if chk and chk[0] != "ok":
                    print("[WARN] PRAGMA quick_check:", chk[0])
            except Exception:
                pass

            return

        except sqlite3.OperationalError as e:
            msg = str(e).lower()
            if "database is locked" in msg or "database busy" in msg:
                if con:
                    try: con.rollback()
                    except: pass
                    try: con.close()
                    except: pass
                if attempt == max_tries:
                    raise
                time.sleep(delay)
                delay = min(delay * 1.8, 3.0)
                continue
            else:
                raise
        finally:
            if con:
                try: con.close()
                except: pass

In [12]:
# === [셀 8] 수집기 ===

def collect_to_sqlite(
    db_path: str,
    keyword: Optional[str] = None,
    per_page: int = DEFAULT_PER_PAGE,
    limit: Optional[int] = None,
    warm_pages: int = 3,
) -> int:
    """
    - keyword가 있으면: 원격 키워드 시도 → 실패 시 무필터 일부 페이지 + 로컬 필터
    - keyword가 없으면: 전수 페이징
    - limit가 있으면 그 개수까지만
    """
    if not SERVICE_KEY:
        raise ValueError("환경변수 ODCLOUD_SERVICE_KEY가 없습니다.")
    init_db(db_path)

    batch: List[Dict[str, Any]] = []
    scanned = 0
    emitted = 0

    it = iter_service_list(keyword=keyword, per_page=per_page, warm_pages=warm_pages, limit=limit)
    for svc in tqdm(it, desc="listing"):
        scanned += 1
        sid = str(svc.get("서비스ID"))
        if not sid:
            continue
        d = fetch_service_detail(sid)
        c = fetch_service_cond(sid)
        row = _row_from_item(svc, d, c)
        batch.append(row)
        emitted += 1
        # 배치 업서트
        if len(batch) >= 200:
            upsert_rows(db_path, batch)
            batch.clear()
        if limit and emitted >= limit:
            break

    if batch:
        upsert_rows(db_path, batch)

    return emitted


In [13]:
# === [셀 9] 실행 예시 ===
db_path = "gov24.sqlite"

# 1) 전체 중 워밍업 페이지만 빠르게(키워드 X) — 개략 동작 확인
n1 = collect_to_sqlite(db_path, keyword=None, per_page=200, limit=300, warm_pages=3)
print("워밍업 수집 건수:", n1)

# 2) 키워드 기반(예: '청년') — 원격 검색 또는 로컬 키워드 필터
n2 = collect_to_sqlite(db_path, keyword="청년", per_page=200, limit=300, warm_pages=5)
print("키워드 '청년' 수집 건수:", n2)


listing: 299it [00:21, 13.98it/s]


워밍업 수집 건수: 300


listing: 37it [00:04,  9.18it/s]

키워드 '청년' 수집 건수: 37


In [14]:
# === [셀 10] 빠른 점검 (수정본 v2) ===
import sqlite3

con = sqlite3.connect("gov24.sqlite")
cur = con.cursor()

print("총 서비스:", cur.execute("SELECT COUNT(*) FROM services;").fetchone()[0])

print("\n샘플 5건(제목/기관/기간/금액):")
for row in cur.execute("""
    SELECT s.title, s.org_name, s.apply_from, s.apply_to, s.min_amount, s.max_amount
    FROM services AS s
    ORDER BY (s.updated_at IS NULL), s.updated_at DESC
    LIMIT 5;
"""):
    print(row)

print("\nFTS 검색(청년): 상위 5건")
q = "청년"
for row in cur.execute("""
    SELECT s.title
    FROM services AS s
    JOIN services_fts AS f ON s.rowid = f.rowid
    WHERE services_fts MATCH ?
    LIMIT 5;
""", (q,)):
    print("-", row[0])

print("\nFTS 검색(청년, bm25 정렬): 상위 5건")
try:
    for row in cur.execute("""
        SELECT s.title, bm25(services_fts) AS score
        FROM services AS s
        JOIN services_fts ON s.rowid = services_fts.rowid
        WHERE services_fts MATCH ?
        ORDER BY score
        LIMIT 5;
    """, (q,)):
        print(f"- {row[0]} (bm25={row[1]:.3f})")
except sqlite3.OperationalError:
    print("- 이 SQLite 빌드는 bm25를 지원하지 않습니다. 위의 기본 검색만 사용하세요.")

con.close()

총 서비스: 329

샘플 5건(제목/기관/기간/금액):
('맞춤형 기술파트너 지원', '중소벤처기업부', None, None, 30000000, 30000000)
('재창업기업 사업화 지원', '중소벤처기업부', None, None, 100000000, 100000000)
('1인 창조기업 지원센터', '중소벤처기업부', None, None, None, None)
('중장년 기술창업센터', '중소벤처기업부', None, None, None, None)
('원스톱기업애로종합지원', '중소벤처기업부', None, None, None, None)

FTS 검색(청년): 상위 5건
- 일상돌봄 서비스
- 문화로 치유 지원
- 청년 문화예술패스 지원
- 산림복지진흥원이 운영하는 산림복지시설의 산림복지소외자 지원
- 청년 일경험 지원

FTS 검색(청년, bm25 정렬): 상위 5건
- 은평자준청 운영 (bm25=-4.905)
- 청년 창업인의 집 운영 (bm25=-4.901)
- 청년 행정체험단 (bm25=-4.647)
- 청년일경험 지원 (bm25=-4.393)
- 귀농귀촌종합센터 운영 (bm25=-4.294)


In [15]:
import pandas as pd
import sqlite3

# 판다스 출력 옵션 설정 (표가 잘리지 않게)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

db_path = "gov24.sqlite"
query = "SELECT * FROM services;"

con = None  # 연결 변수 초기화
try:
    # 데이터베이스에 연결
    con = sqlite3.connect(db_path)

    # SQL 쿼리를 실행하고 결과를 판다스 데이터프레임으로 로드
    df = pd.read_sql_query(query, con)

finally:
    # 작업이 끝나면 항상 연결을 닫아줍니다.
    if con:
        con.close()

# 데이터프레임을 출력합니다. (노트북 셀의 마지막 줄에 변수만 두면 자동으로 표가 그려집니다)
df

,service_id,title,purpose_summary,category,org_name,org_type,org_code,applicant_type,support_type,support_desc,target_desc,apply_method,apply_period,apply_from,apply_to,phone,detail_url,created_at,updated_at,views,min_amount,max_amount,raw,cond_target,cond_selection,cond_age,cond_income,cond_residence,cond_eligibility
0,000000465790,유아학비 (누리과정) 지원,"유치원에 다니는 만 3~5세 아동에게 유아학비, 방과후과정비 등 지원",보육·교육,교육부,중앙행정기관,1342000,개인,현금(감면),"○ 3~5세에 대해 교육비를 지급합니다. - 국공립 100,000원, 사립 280,...",○ 지원대상 : 국공립 및 사립유치원에 다니는 3~5세 유아 - '22년 1~2월생...,기타 온라인신청||방문신청,상시신청,None,None,교육부/02-6222-6060||0079에듀콜/1544-0079-5-1,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/0000...,20201217142613,20250805161216,224604,5.000000e+04,2.800000e+05,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""영유...",○ 지원대상 : 국공립 및 사립유치원에 다니는 3~5세 유아 - '22년 1~2월생...,※ 2025. 3. 1~2026.2.28. 까지 적용 ○ 지원대상 : 국공립유치원 ...,3세\n4세\n5세,None,None,None
1,105100000001,근로·자녀장려금,"소득과 재산이 적은 근로소득자에게 근로장려금을, 자녀가 있을 경우 자녀장려금 지급",주거·자립,국세청,중앙행정기관,1210000,가구,현금,"○ 전년도 연간 부부합산 총 급여액 등(근로소득, 사업소득 또는 종교인소득의 합계)...","○ 신청요건을 모두 충족하는 근로소득, 사업소득 또는 종교인소득이 있는 가구로써 신...",기타 온라인신청,○ 정기신청 : 5.1.~5.31.○ 반기신청 - 상반기분 신청 : 9.1.~9.1...,None,None,해당지역 지자체(세무서)/관할 세무서,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1051...,20201217142613,20250717101438,1691373,1.000000e+06,3.300000e+06,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""장려...","○ 신청요건을 모두 충족하는 근로소득, 사업소득 또는 종교인소득이 있는 가구로써 신...","○ 아래 요건을 모두 충족하는 근로소득, 사업소득 또는 종교인소득이 있는 가구 - ...",18세\n70세 이상,"급여액 등(근로소득, 사업소득 또는 종교인소득의 합계)에 따라 - 근로장려금은 ㆍ ...",해당지역 지자체(세무서)/관할 세무서,None
2,116010000001,주택금융공사 월세 자금보증,일정요건의 월세대출 대상자에게 월세자금보증을 지원,주거·자립,한국주택금융공사,공공기관,B551408,개인,상담/법률지원||현금(융자),○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원 ...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,방문신청,주택도시기금 주거안정 월세대출 규정에 따름,None,None,주택금융공사/1688-8114,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1160...,20201217142613,20241018141915,31236,NaN,NaN,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""주택...",○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,만 35세,None,None,None
3,119200000001,친환경 에너지절감장비 보급,"연근해 어업인을 대상으로 노후기관, 장비, 설비를 대체 보급",농림축산어업,해양수산부,중앙행정기관,1192000,개인,현금,"○ 고효율 등(燈)(LED, 무전극등(燈) 등) ○ 노후화된 기관(디젤, 가솔린기관...","○ 어선의 기관, 고효율 등(燈)(집어등, 작업등) 외 에너지 절감 장비의 대체 또...",방문신청,연초 모집공고에 따름,None,None,해양수산부 수산정책실 어선안전정책과/044-200-5528,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1192...,20201217142613,20250725163624,2713,NaN,NaN,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""어선...","○ 어선의 기관, 고효율 등(燈)(집어등, 작업등) 외 에너지 절감 장비의 대체 또...",○ 우선순위 가. 1순위 : 고효율등(燈) 및 유류절감장치 등 나. 2순위 : 디젤...,None,None,None,None
4,119200000007,해양사고 국선 심판변론인 선정 지원,해양사고를 입은 사회적 약자에게 국선 심판변론인 선정 및 법률자문 지원,행정·안전,해양수산부,중앙행정기관,1192000,개인,기타||상담/법률지원,○ 해양사고관련자가 심판원에 대하여 하는 신청ㆍ청구ㆍ진술 등의 대리 또는 대행 ○ ...,"○ 사회적 약자(연령, 장애, 빈곤, 교육정도 등)에 해당하는 해양사고관련자",방문신청,해당사건 접수후,None,None,해양수산부 중앙해양안전심판원/044-200-6117,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1192...,20201217142613,20250723114305,1767,NaN,NaN,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""심판...","○ 사회적 약자(연령, 장애, 빈곤, 교육정도 등)에 해당하는 해양사고관련자",○ 다음 각각 호의 어느 하나에 해당하는 경우로서 심판변론인이 없는 때 ○ 국선 심...,70세 이상,소득(4인가구 기준) 60%이하,None,None
5,119200000008,옵서버 승선경비 지원,국제옵서버를 승선시킨 원양선사에 승선경비 및 활동 등을 지원,농림축산어업,해양수산부,중앙행정기관,1192000,법인/시설/단체,현금,○ 원양어선에 승선하여 활동하는 국제옵서버 승선경비 및 활동 지원,○ 국제옵서버를 승선시킨 원양선사,방문신청,수시,None,None,한국수산자원공단 TAC관리실/051-718-2482,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1192...,20201217142613,20250723161419,1438,NaN,NaN,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""원양...",○ 국제옵서버를 승선시킨 원양선사,○ 국제옵서버를 승선시킨 원양선사 1) 한국수산자원공단에 국제옵서버 일정하여 선사부...,None,None,None,None
6,119200000010,합작수산물 관세 감면 추진,적법하게 채포한 수산물을 국내로 수입하는 자에게 관세감면,농림축산어업,해양수산부,중앙행정기관,1192000,법인/시설/단체,현금(감면),○ 관세감면,○ 기획재정부령에서 정한 방법 또는 요건에 적합하게 채포한 수산물을 국내로 수입하는 자,기타 온라인신청,수시,None,None,해양수산부/044-200-5364,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1192...,20201217142613,20250723094847,1606,NaN,NaN,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""원양...",○ 기획재정

In [16]:
import pandas as pd
import sqlite3

# --- 1. 데이터베이스에서 데이터를 DataFrame으로 다시 로드 ---

db_path = "gov24.sqlite"
query = "SELECT * FROM services;"

con = None
try:
    con = sqlite3.connect(db_path)
    df = pd.read_sql_query(query, con)
finally:
    if con:
        con.close()

# --- 2. DataFrame을 CSV 파일로 저장 ---

output_filename = "gov24_services.csv"

# to_csv 메서드를 사용하여 저장합니다.
df.to_csv(
    output_filename,          # 저장할 파일 이름
    encoding='utf-8-sig',     # 엑셀에서 한글이 깨지지 않게 하는 중요한 옵션
    index=False               # 데이터프레임의 인덱스는 저장하지 않음
)

print(f"✅ 데이터가 '{output_filename}' 파일로 성공적으로 저장되었습니다.")
print(f"저장된 데이터 행의 수: {len(df)}개")

# --- 3. 저장된 데이터의 첫 5줄 미리보기 ---
print("\n[저장된 데이터 샘플]")
df.head()

✅ 데이터가 'gov24_services.csv' 파일로 성공적으로 저장되었습니다.
저장된 데이터 행의 수: 329개

[저장된 데이터 샘플]


,service_id,title,purpose_summary,category,org_name,org_type,org_code,applicant_type,support_type,support_desc,target_desc,apply_method,apply_period,apply_from,apply_to,phone,detail_url,created_at,updated_at,views,min_amount,max_amount,raw,cond_target,cond_selection,cond_age,cond_income,cond_residence,cond_eligibility
0,000000465790,유아학비 (누리과정) 지원,"유치원에 다니는 만 3~5세 아동에게 유아학비, 방과후과정비 등 지원",보육·교육,교육부,중앙행정기관,1342000,개인,현금(감면),"○ 3~5세에 대해 교육비를 지급합니다. - 국공립 100,000원, 사립 280,...",○ 지원대상 : 국공립 및 사립유치원에 다니는 3~5세 유아 - '22년 1~2월생...,기타 온라인신청||방문신청,상시신청,None,None,교육부/02-6222-6060||0079에듀콜/1544-0079-5-1,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/0000...,20201217142613,20250805161216,224604,50000.0,280000.0,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""영유...",○ 지원대상 : 국공립 및 사립유치원에 다니는 3~5세 유아 - '22년 1~2월생...,※ 2025. 3. 1~2026.2.28. 까지 적용 ○ 지원대상 : 국공립유치원 ...,3세\n4세\n5세,None,None,None
1,105100000001,근로·자녀장려금,"소득과 재산이 적은 근로소득자에게 근로장려금을, 자녀가 있을 경우 자녀장려금 지급",주거·자립,국세청,중앙행정기관,1210000,가구,현금,"○ 전년도 연간 부부합산 총 급여액 등(근로소득, 사업소득 또는 종교인소득의 합계)...","○ 신청요건을 모두 충족하는 근로소득, 사업소득 또는 종교인소득이 있는 가구로써 신...",기타 온라인신청,○ 정기신청 : 5.1.~5.31.○ 반기신청 - 상반기분 신청 : 9.1.~9.1...,None,None,해당지역 지자체(세무서)/관할 세무서,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1051...,20201217142613,20250717101438,1691373,1000000.0,3300000.0,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""장려...","○ 신청요건을 모두 충족하는 근로소득, 사업소득 또는 종교인소득이 있는 가구로써 신...","○ 아래 요건을 모두 충족하는 근로소득, 사업소득 또는 종교인소득이 있는 가구 - ...",18세\n70세 이상,"급여액 등(근로소득, 사업소득 또는 종교인소득의 합계)에 따라 - 근로장려금은 ㆍ ...",해당지역 지자체(세무서)/관할 세무서,None
2,116010000001,주택금융공사 월세 자금보증,일정요건의 월세대출 대상자에게 월세자금보증을 지원,주거·자립,한국주택금융공사,공공기관,B551408,개인,상담/법률지원||현금(융자),○ 주택도시기금 주거안정 월세대출 요건을 충족하는 대상자에 대해 월세자금보증 지원 ...,○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,방문신청,주택도시기금 주거안정 월세대출 규정에 따름,None,None,주택금융공사/1688-8114,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1160...,20201217142613,20241018141915,31236,NaN,NaN,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""주택...",○ 보증금 1억 원 이하 및 월세금 60만 원 이하인 월세 계약을 체결한 주택도시 ...,○ (주택도시기금 주거안정 월세대출 대상자 요건) 무주택 세대주(세대원 전원 포함)...,만 35세,None,None,None
3,119200000001,친환경 에너지절감장비 보급,"연근해 어업인을 대상으로 노후기관, 장비, 설비를 대체 보급",농림축산어업,해양수산부,중앙행정기관,1192000,개인,현금,"○ 고효율 등(燈)(LED, 무전극등(燈) 등) ○ 노후화된 기관(디젤, 가솔린기관...","○ 어선의 기관, 고효율 등(燈)(집어등, 작업등) 외 에너지 절감 장비의 대체 또...",방문신청,연초 모집공고에 따름,None,None,해양수산부 수산정책실 어선안전정책과/044-200-5528,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1192...,20201217142613,20250725163624,2713,NaN,NaN,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""어선...","○ 어선의 기관, 고효율 등(燈)(집어등, 작업등) 외 에너지 절감 장비의 대체 또...",○ 우선순위 가. 1순위 : 고효율등(燈) 및 유류절감장치 등 나. 2순위 : 디젤...,None,None,None,None
4,119200000007,해양사고 국선 심판변론인 선정 지원,해양사고를 입은 사회적 약자에게 국선 심판변론인 선정 및 법률자문 지원,행정·안전,해양수산부,중앙행정기관,1192000,개인,기타||상담/법률지원,○ 해양사고관련자가 심판원에 대하여 하는 신청ㆍ청구ㆍ진술 등의 대리 또는 대행 ○ ...,"○ 사회적 약자(연령, 장애, 빈곤, 교육정도 등)에 해당하는 해양사고관련자",방문신청,해당사건 접수후,None,None,해양수산부 중앙해양안전심판원/044-200-6117,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/1192...,20201217142613,20250723114305,1767,NaN,NaN,"{""list"": {""등록일시"": ""20201217142613"", ""부서명"": ""심판...","○ 사회적 약자(연령, 장애, 빈곤, 교육정도 등)에 해당하는 해양사고관련자",○ 다음 각각 호의 어느 하나에 해당하는 경우로서 심판변론인이 없는 때 ○ 국선 심...,70세 이상,소득(4인가구 기준) 60%이하,None,None


In [ ]:
# OpenAI API 설치 및 임포트
!pip install openai

import openai
import json
import time
from typing import Dict, Any, List, Optional

# OpenAI API 키 설정 (여기에 본인의 API 키를 입력하세요)
# openai.api_key = "your-openai-api-key-here"
# 또는 환경변수로 설정
import os
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"
openai.api_key = os.getenv("OPENAI_API_KEY")


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [22]:
def normalize_conditions_with_openai(text: str, field_type: str) -> Dict[str, Any]:
    """
    OpenAI API를 사용하여 서술형 조건을 구조화된 형태로 변환
    """
    
    if not text or pd.isna(text):
        return {}
    
    # 필드별 프롬프트 정의 (구체적 조건 보존)
    prompts = {
        "age": """
다음 나이 조건을 분석하여 표준화된 형태로 변환해주세요.

원본: {text}

다음 JSON 형식으로만 응답해주세요:
{{
    "min_age": 숫자 또는 null,
    "max_age": 숫자 또는 null,
    "age_unit": "세",
    "age_restriction": "제한없음" 또는 "있음"
}}
""",
        
        "income": """
다음 소득 조건을 분석하여 표준화된 형태로 변환해주세요.

원본: {text}

다음 JSON 형식으로만 응답해주세요:
{{
    "income_type": "중위소득" 또는 "기준중위소득" 또는 "기타" 또는 null,
    "income_percentage": 숫자 또는 null,
    "income_amount": 숫자 또는 null,
    "income_unit": "원" 또는 "만원" 또는 null,
    "income_restriction": "제한없음" 또는 "있음"
}}
""",
        
        "target": """
다음 지원대상을 분석하여 표준화된 형태로 변환해주세요.

원본: {text}

다음 JSON 형식으로만 응답해주세요:
{{
    "target_age_group": "청년" 또는 "중장년" 또는 "노인" 또는 "전연령" 또는 null,
    "target_employment": "취업자" 또는 "구직자" 또는 "창업자" 또는 "학생" 또는 "무관" 또는 null,
    "target_education": "초졸" 또는 "중졸" 또는 "고졸" 또는 "대졸" 또는 "대학원" 또는 "무관" 또는 null,
    "target_household": "1인가구" 또는 "2인가구" 또는 "3인가구" 또는 "4인가구" 또는 "5인이상" 또는 "무관" 또는 null,
    "target_special": ["장애인", "한부모", "다문화", "북한이탈주민", "기초생활수급자", "차상위계층", "기타"] 또는 [],
    "target_occupation": ["농업인", "어업인", "자영업자", "프리랜서", "기타"] 또는 [],
    "target_specific_conditions": "구체적인 지원 요건이나 특수 조건" 또는 null,
    "target_note": "추가 설명 또는 null"
}}

중요: target_specific_conditions에는 구체적인 업종, 시설, 장비, 계약 조건 등 상세한 내용을 포함해주세요.

예시:
- "어선의 기관, 어선인" → {{
    "target_occupation": ["어업인"],
    "target_specific_conditions": "어선의 기관, 어선인",
    "target_note": null
}}

- "보증급 1억원 이하 및 월세금 60만원 이하인 월세 계약을 체결한 주택도시 기금 주거안정 월세대출 대상자" → {{
    "target_employment": "무관",
    "target_specific_conditions": "보증급 1억원 이하 및 월세금 60만원 이하인 월세 계약을 체결한 주택도시 기금 주거안정 월세대출 대상자",
    "target_note": null
}}
""",
        
        "selection": """
다음 선정기준을 분석하여 표준화된 형태로 변환해주세요.

원본: {text}

다음 JSON 형식으로만 응답해주세요:
{{
    "selection_method": "서류심사" 또는 "면접" 또는 "추첨" 또는 "선착순" 또는 "종합평가" 또는 "기타",
    "required_documents": ["서류1", "서류2"] 또는 [],
    "evaluation_criteria": ["기준1", "기준2"] 또는 [],
    "selection_specific_conditions": "구체적인 선정 조건이나 특수 요건" 또는 null,
    "selection_note": "추가 설명 또는 null"
}}

중요: selection_specific_conditions에는 구체적인 선정 기준, 우선순위, 제외조건 등을 포함해주세요.
""",
        
        "residence": """
다음 거주지 조건을 분석해주세요.

원본: {text}

다음 JSON 형식으로만 응답해주세요:
{{
    "residence_type": "전국" 또는 "특정지역" 또는 "제한없음",
    "specific_regions": ["지역명"] 또는 [],
    "residence_duration": 숫자 또는 null,
    "residence_duration_unit": "년" 또는 "개월" 또는 null,
    "residence_specific_conditions": "구체적인 거주 조건이나 특수 요건" 또는 null,
    "residence_note": "추가 설명 또는 null"
}}

중요: residence_specific_conditions에는 구체적인 거주 형태, 주택 유형, 전입 조건 등을 포함해주세요.
"""
    }
    
    try:
        prompt = prompts.get(field_type, "").format(text=text)
        
        # 새로운 OpenAI API 인터페이스 사용
        from openai import OpenAI
        client = OpenAI(api_key=openai.api_key)
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "당신은 복지 정책 데이터를 정규화하는 전문가입니다. 구체적인 조건들을 보존하면서 정확한 JSON 형식으로 응답해주세요."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=800,  # 더 긴 응답 허용
            temperature=0.1
        )
        
        result_text = response.choices[0].message.content.strip()
        
        # JSON 파싱
        try:
            result = json.loads(result_text)
            return result
        except json.JSONDecodeError:
            # JSON 추출 시도
            import re
            json_match = re.search(r'\{.*\}', result_text, re.DOTALL)
            if json_match:
                return json.loads(json_match.group())
            else:
                return {"error": "JSON 파싱 실패", "raw_response": result_text}
                
    except Exception as e:
        print(f"API 호출 오류 ({field_type}): {e}")
        return {"error": str(e), "original_text": text}

In [23]:
def create_clean_normalized_csv(df_original):
    """서술형 데이터를 완전히 새로운 깔끔한 CSV로 변환"""
    
    clean_data = []
    total_rows = len(df_original)
    
    print(f"🧹 깔끔한 CSV 생성 시작: {total_rows}개 행")
    print("원본 서술형 데이터를 구조화된 형태로 변환합니다...")
    
    for idx, row in tqdm(df_original.iterrows(), total=total_rows, desc="데이터 정규화"):
        try:
            # 기본 정보
            clean_row = {
                'service_id': row['service_id'],
                'title': row['title'],
                'purpose_summary': row['purpose_summary'],
                'category': row['category'],
                'org_name': row['org_name'],
                'org_type': row['org_type'],
                'applicant_type': row['applicant_type'],
                'support_type': row['support_type'],
                'support_desc': row['support_desc'],
                'apply_method': row['apply_method'],
                'apply_period': row['apply_period'],
                'min_amount': row['min_amount'],
                'max_amount': row['max_amount'],
                'phone': row['phone'],
                'detail_url': row['detail_url']
            }
            
            # === 나이 조건 정규화 ===
            if pd.notna(row['cond_age']) and str(row['cond_age']).strip():
                age_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_age']), 
                    'age'
                )
                
                clean_row.update({
                    'age_min': age_data.get('min_age'),
                    'age_max': age_data.get('max_age'),
                    'age_unit': age_data.get('age_unit'),
                    'age_note': age_data.get('age_note'),
                    'age_restriction': age_data.get('age_restriction')
                })
            else:
                clean_row.update({
                    'age_min': None,
                    'age_max': None,
                    'age_unit': None,
                    'age_note': None,
                    'age_restriction': '제한없음'
                })
            
            # === 소득 조건 정규화 ===
            if pd.notna(row['cond_income']) and str(row['cond_income']).strip():
                income_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_income']), 
                    'income'
                )
                
                clean_row.update({
                    'income_type': income_data.get('income_type'),
                    'income_percentage': income_data.get('income_percentage'),
                    'income_amount': income_data.get('income_amount'),
                    'income_unit': income_data.get('income_unit'),
                    'income_note': income_data.get('income_note'),
                    'income_restriction': income_data.get('income_restriction')
                })
            else:
                clean_row.update({
                    'income_type': None,
                    'income_percentage': None,
                    'income_amount': None,
                    'income_unit': None,
                    'income_note': None,
                    'income_restriction': '제한없음'
                })
            
            # === 지원대상 정규화 (구체적 조건 보존) ===
            if pd.notna(row['cond_target']) and str(row['cond_target']).strip():
                target_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_target']), 
                    'target'
                )
                
                clean_row.update({
                    'target_age_group': target_data.get('target_age_group'),
                    'target_employment': target_data.get('target_employment'),
                    'target_education': target_data.get('target_education'),
                    'target_household': target_data.get('target_household'),
                    'target_special': target_data.get('target_special'),
                    'target_occupation': target_data.get('target_occupation'),
                    'target_specific_conditions': target_data.get('target_specific_conditions'),  # 새로 추가!
                    'target_note': target_data.get('target_note')
                })
            else:
                clean_row.update({
                    'target_age_group': None,
                    'target_employment': None,
                    'target_education': None,
                    'target_household': None,
                    'target_special': [],
                    'target_occupation': [],
                    'target_specific_conditions': None,  # 새로 추가!
                    'target_note': None
                })
            
            # === 선정기준 정규화 (구체적 조건 보존) ===
            if pd.notna(row['cond_selection']) and str(row['cond_selection']).strip():
                selection_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_selection']), 
                    'selection'
                )
                
                clean_row.update({
                    'selection_method': selection_data.get('selection_method'),
                    'required_documents': selection_data.get('required_documents'),
                    'evaluation_criteria': selection_data.get('evaluation_criteria'),
                    'selection_specific_conditions': selection_data.get('selection_specific_conditions'),  # 새로 추가!
                    'selection_note': selection_data.get('selection_note')
                })
            else:
                clean_row.update({
                    'selection_method': None,
                    'required_documents': [],
                    'evaluation_criteria': [],
                    'selection_specific_conditions': None,  # 새로 추가!
                    'selection_note': None
                })
            
            # === 거주지 조건 정규화 (구체적 조건 보존) ===
            if pd.notna(row['cond_residence']) and str(row['cond_residence']).strip():
                residence_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_residence']), 
                    'residence'
                )
                
                clean_row.update({
                    'residence_type': residence_data.get('residence_type'),
                    'specific_regions': residence_data.get('specific_regions'),
                    'residence_duration': residence_data.get('residence_duration'),
                    'residence_duration_unit': residence_data.get('residence_duration_unit'),
                    'residence_specific_conditions': residence_data.get('residence_specific_conditions'),  # 새로 추가!
                    'residence_note': residence_data.get('residence_note')
                })
            else:
                clean_row.update({
                    'residence_type': '전국',
                    'specific_regions': [],
                    'residence_duration': None,
                    'residence_duration_unit': None,
                    'residence_specific_conditions': None,  # 새로 추가!
                    'residence_note': None
                })
            
            clean_data.append(clean_row)
            
            # 진행상황 출력 (10개마다)
            if (idx + 1) % 10 == 0:
                print(f"✅ {idx + 1}/{total_rows} 완료")
                
        except Exception as e:
            print(f"❌ 행 {idx} 처리 중 오류: {e}")
            clean_data.append(clean_row)
            continue
    
    print(f"\\n🎉 정규화 완료! 총 {len(clean_data)}개 행 처리됨")
    return pd.DataFrame(clean_data)

In [24]:
# CSV 파일 읽기
print("기존 CSV 파일 읽는 중...")
df_original = pd.read_csv('gov24_services.csv')

print(f"원본 데이터: {len(df_original)}개 행")
print("\\n📋 처리할 샘플 데이터:")
sample_data = df_original[['title', 'cond_age', 'cond_income', 'cond_target']].head(3)
for idx, row in sample_data.iterrows():
    print(f"\\n{idx+1}. {row['title']}")
    print(f"   나이: {row['cond_age']}")
    print(f"   소득: {row['cond_income']}")
    print(f"   대상: {row['cond_target']}")

# 소량 테스트 실행 (3개만)
print("\\n=== 🧪 소량 테스트 시작 ===")
print("완전히 새로운 구조의 깔끔한 CSV를 생성합니다...")

df_clean = create_clean_normalized_csv(df_original.head(3))

# 새로운 깔끔한 CSV 저장
clean_filename = 'gov24_services_clean_normalized.csv'
df_clean.to_csv(clean_filename, index=False, encoding='utf-8-sig')

print(f"\\n✅ 깔끔한 CSV 저장: {clean_filename}")

# 결과 확인
print("\\n📊 새로운 CSV 구조:")
print(f"총 컬럼 수: {len(df_clean.columns)}개")
print("\\n새로운 컬럼들:")
for col in df_clean.columns:
    print(f"  - {col}")

# 샘플 데이터 확인
print("\\n�� 정규화 결과 샘플:")
print(df_clean[['title', 'age_min', 'age_max', 'income_type', 'income_percentage', 'target_age_group']].head())

기존 CSV 파일 읽는 중...
원본 데이터: 329개 행
\n📋 처리할 샘플 데이터:
\n1. 유아학비 (누리과정) 지원
   나이: 3세
4세
5세
   소득: nan
   대상: ○ 지원대상 : 국공립 및 사립유치원에 다니는 3~5세 유아 - '22년 1~2월생으로 유치원 입학을 희망하여 3세반에 취원한 유아도 지원 대상 - 취학대상 아동('18.1.1~12.31.출생)이 취학을 유예하는 경우, 유예한 1년에 한하여 5세 유아 무상교육비 지원(취학유예 통지서 제출) ※ 단, 지원기간은 3년을 초과할 수 없음. ○ 추가지원 : 저소득층 유아(유아학비 지원 대상 자격이 있고, 사립유치원에 다니는 법정저소득층(기초생활수급자, 차상위계층, 한부모 가정) 유아) ○ 아래의 경우 지원대상에서 제외 - 대한민국 국적을 가지지 않은 유아(난민 및 「재한외국인 처우 기본법」에 따라 법무부장관이 인정한 '특별기여자 등'은 예외적으로 인정) - 가정 양육수당 및 어린이집 보육료를 지원 받고 있는 유아 - 유치원 이용시간에 아이돌봄서비스 등과 중복지원 불가 - 해외 체류 기간이 31일째 되는 날 유아학비 지원자격 중지 ○ 자격 중지 후 유아학비를 다시 지원받기 위해서는 재신청 필요, 신청 누락으로 발생되는 지원금은 소급지원 되지 않음.
\n2. 근로·자녀장려금
   나이: 18세
70세 이상
   소득: 급여액 등(근로소득, 사업소득 또는 종교인소득의 합계)에 따라 - 근로장려금은 ㆍ 단독가구 최대 165만 원 
소득, 사업소득 또는 종교인소득이 있는 가구 - 소득요건 : 전년도 부부합산 연간 총소득이 가구 유형에 따라 정한 총 소득기준금액 미만일 것 (근로장려금) ㆍ 단독가구 : 2,200만 원 미만
소득금액 100만원 이하
재산요건 ㆍ 전년도 6월 1일 현재, 가구원 전체의 재산 합계액이 2억4천만 원 미만일 것 - 가구요건 ㆍ 단독가구 : 배우자1), 부양자녀2), 70세 이상 직계존속3)이 모두 없는 가구 ㆍ 홑벌이가구 : 배우자 또는 부양자녀 또는 70세 이상 직계존

데이터 정규화: 100%|██████████| 3/3 [00:40<00:00, 13.54s/it]

\n🎉 정규화 완료! 총 3개 행 처리됨
\n✅ 깔끔한 CSV 저장: gov24_services_clean_normalized.csv
\n📊 새로운 CSV 구조:
총 컬럼 수: 45개
\n새로운 컬럼들:
  - service_id
  - title
  - purpose_summary
  - category
  - org_name
  - org_type
  - applicant_type
  - support_type
  - support_desc
  - apply_method
  - apply_period
  - min_amount
  - max_amount
  - phone
  - detail_url
  - age_min
  - age_max
  - age_unit
  - age_note
  - age_restriction
  - income_type
  - income_percentage
  - income_amount
  - income_unit
  - income_note
  - income_restriction
  - target_age_group
  - target_employment
  - target_education
  - target_household
  - target_special
  - target_occupation
  - target_specific_conditions
  - target_note
  - selection_method
  - required_documents
  - evaluation_criteria
  - selection_specific_conditions
  - selection_note
  - residence_type
  - specific_regions
  - residence_duration
  - residence_duration_unit
  - residence_specific_conditions
  - residence_note
\n�� 정규화 결과 샘플:
            title  age_mi

In [25]:
def process_full_dataset_clean():
    """전체 데이터셋을 깔끔한 CSV로 변환"""
    
    print("🚀 전체 데이터셋 처리 시작...")
    print(f"총 {len(df_original)}개 행을 처리합니다.")
    
    # 전체 데이터 처리
    df_full_clean = create_clean_normalized_csv(df_original)
    
    # 최종 저장
    final_filename = 'gov24_services_final_clean.csv'
    df_full_clean.to_csv(final_filename, index=False, encoding='utf-8-sig')
    
    print(f"\\n🎉 최종 깔끔한 CSV 저장 완료: {final_filename}")
    print(f"총 {len(df_full_clean)}개 행, {len(df_full_clean.columns)}개 컬럼")
    
    # 결과 요약
    print("\\n📊 정규화 결과 요약:")
    print(f"- 나이 조건이 있는 정책: {df_full_clean['age_min'].notna().sum()}개")
    print(f"- 소득 조건이 있는 정책: {df_full_clean['income_percentage'].notna().sum()}개")
    print(f"- 지원대상이 있는 정책: {df_full_clean['target_age_group'].notna().sum()}개")
    
    return df_full_clean

# 함수 실행
print("=== 🚀 전체 데이터 처리 시작 ===")
try:
    result = process_full_dataset_clean()
    print("\\n✅ 전체 처리 완료!")
    print(f"결과 데이터 크기: {len(result)} 행 x {len(result.columns)} 컬럼")
    
    # 샘플 데이터 확인
    print("\\n 정규화 결과 샘플:")
    sample_cols = ['title', 'age_min', 'age_max', 'income_type', 'income_percentage', 'target_age_group']
    print(result[sample_cols].head())
    
except Exception as e:
    print(f"\\n❌ 오류 발생: {e}")
    import traceback
    traceback.print_exc()

=== 🚀 전체 데이터 처리 시작 ===
🚀 전체 데이터셋 처리 시작...
총 329개 행을 처리합니다.
🧹 깔끔한 CSV 생성 시작: 329개 행
원본 서술형 데이터를 구조화된 형태로 변환합니다...


데이터 정규화:   3%|▎         | 10/329 [01:42<45:56,  8.64s/it] 

✅ 10/329 완료


데이터 정규화:   6%|▌         | 20/329 [02:56<37:19,  7.25s/it]

✅ 20/329 완료


...

✅ 320/329 완료


데이터 정규화:  98%|█████████▊| 321/329 [48:30<01:22, 10.30s/it]

API 호출 오류 (age): Extra data: line 7 column 1 (char 91)


데이터 정규화: 100%|██████████| 329/329 [49:26<00:00,  9.02s/it]

\n🎉 정규화 완료! 총 329개 행 처리됨
\n🎉 최종 깔끔한 CSV 저장 완료: gov24_services_final_clean.csv
총 329개 행, 45개 컬럼
\n📊 정규화 결과 요약:
- 나이 조건이 있는 정책: 70개
- 소득 조건이 있는 정책: 27개
- 지원대상이 있는 정책: 89개
\n✅ 전체 처리 완료!
결과 데이터 크기: 329 행 x 45 컬럼
\n 정규화 결과 샘플:
                 title  age_min  age_max income_type  income_percentage  \
0       유아학비 (누리과정) 지원      3.0      5.0        None                NaN   
1             근로·자녀장려금     18.0     70.0          기타                NaN   
2       주택금융공사 월세 자금보증     35.0     35.0        None                NaN   
3       친환경 에너지절감장비 보급      NaN      NaN        None                NaN   
4  해양사고 국선 심판변론인 선정 지원     70.0      NaN        None               60.0   

  target_age_group  
0             None  
1             None  
2             None  
3             None  
4             None  


In [27]:
def create_clean_normalized_csv(df_original):
    """서술형 데이터를 완전히 새로운 깔끔한 CSV로 변환"""
    
    clean_data = []
    total_rows = len(df_original)
    
    print(f"🧹 깔끔한 CSV 생성 시작: {total_rows}개 행")
    print("원본 서술형 데이터를 구조화된 형태로 변환합니다...")
    
    for idx, row in tqdm(df_original.iterrows(), total=total_rows, desc="데이터 정규화"):
        try:
            # === 1. serviceDetail에서 직접 추출 (정적 정보) ===
            clean_row = {
                'service_id': row['service_id'],
                'title': row['title'],
                'purpose_summary': row['purpose_summary'],
                'category': row['category'],
                'org_name': row['org_name'],
                'org_type': row['org_type'],
                'applicant_type': row['applicant_type'],
                'support_type': row['support_type'],
                'apply_method': row['apply_method'],
                'apply_period': row['apply_period'],
                'min_amount': row['min_amount'],
                'max_amount': row['max_amount'],
                'phone': row['phone'],
                'detail_url': row['detail_url']
            }
            
            # === 2. GPT 모델로 요약 추출 (동적 정보) ===
            
            # 나이 조건
            if pd.notna(row['cond_age']) and str(row['cond_age']).strip():
                age_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_age']), 
                    'age'
                )
                clean_row.update({
                    'age_min': age_data.get('min_age'),
                    'age_max': age_data.get('max_age'),
                    'age_unit': age_data.get('age_unit'),
                    'age_note': age_data.get('age_note'),
                    'age_restriction': age_data.get('age_restriction')
                })
            else:
                clean_row.update({
                    'age_min': None, 'age_max': None, 'age_unit': None,
                    'age_note': None, 'age_restriction': '제한없음'
                })
            
            # 소득 조건
            if pd.notna(row['cond_income']) and str(row['cond_income']).strip():
                income_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_income']), 
                    'income'
                )
                clean_row.update({
                    'income_type': income_data.get('income_type'),
                    'income_percentage': income_data.get('income_percentage'),
                    'income_amount': income_data.get('income_amount'),
                    'income_unit': income_data.get('income_unit'),
                    'income_note': income_data.get('income_note'),
                    'income_restriction': income_data.get('income_restriction')
                })
            else:
                clean_row.update({
                    'income_type': None, 'income_percentage': None,
                    'income_amount': None, 'income_unit': None,
                    'income_note': None, 'income_restriction': '제한없음'
                })
            
            # 지원대상 (target)
            if pd.notna(row['cond_target']) and str(row['cond_target']).strip():
                target_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_target']), 
                    'target'
                )
                clean_row.update({
                    'target_age_group': target_data.get('target_age_group'),
                    'target_employment': target_data.get('target_employment'),
                    'target_education': target_data.get('target_education'),
                    'target_household': target_data.get('target_household'),
                    'target_special': target_data.get('target_special'),
                    'target_occupation': target_data.get('target_occupation'),
                    'target_specific_conditions': target_data.get('target_specific_conditions'),
                    'target_note': target_data.get('target_note')
                })
            else:
                clean_row.update({
                    'target_age_group': None, 'target_employment': None,
                    'target_education': None, 'target_household': None,
                    'target_special': [], 'target_occupation': [],
                    'target_specific_conditions': None, 'target_note': None
                })
            
            # 선정기준 (selection) - GPT로 요약
            if pd.notna(row['cond_selection']) and str(row['cond_selection']).strip():
                selection_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_selection']), 
                    'selection'
                )
                clean_row.update({
                    'selection_method': selection_data.get('selection_method'),
                    'required_documents': selection_data.get('required_documents'),
                    'evaluation_criteria': selection_data.get('evaluation_criteria'),
                    'selection_specific_conditions': selection_data.get('selection_specific_conditions'),
                    'selection_note': selection_data.get('selection_note')
                })
            else:
                clean_row.update({
                    'selection_method': None, 'required_documents': [],
                    'evaluation_criteria': [], 'selection_specific_conditions': None,
                    'selection_note': None
                })
            
            # 거주지 조건 (residence)
            if pd.notna(row['cond_residence']) and str(row['cond_residence']).strip():
                residence_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['cond_residence']), 
                    'residence'
                )
                clean_row.update({
                    'residence_type': residence_data.get('residence_type'),
                    'specific_regions': residence_data.get('specific_regions'),
                    'residence_duration': residence_data.get('residence_duration'),
                    'residence_duration_unit': residence_data.get('residence_duration_unit'),
                    'residence_specific_conditions': residence_data.get('residence_specific_conditions'),
                    'residence_note': residence_data.get('residence_note')
                })
            else:
                clean_row.update({
                    'residence_type': '전국', 'specific_regions': [],
                    'residence_duration': None, 'residence_duration_unit': None,
                    'residence_specific_conditions': None, 'residence_note': None
                })
            
            # 지원내용 요약 (support_desc) - GPT로 요약
            if pd.notna(row['support_desc']) and str(row['support_desc']).strip():
                support_data = api_call_with_delay(
                    normalize_conditions_with_openai, 
                    str(row['support_desc']), 
                    'support'
                )
                clean_row.update({
                    'support_summary': support_data.get('support_summary'),
                    'support_amount_type': support_data.get('support_amount_type'),
                    'support_note': support_data.get('support_note')
                })
            else:
                clean_row.update({
                    'support_summary': None, 'support_amount_type': None, 'support_note': None
                })
            
            clean_data.append(clean_row)
            
            if (idx + 1) % 10 == 0:
                print(f"✅ {idx + 1}/{total_rows} 완료")
                
        except Exception as e:
            print(f"❌ 행 {idx} 처리 중 오류: {e}")
            clean_data.append(clean_row)
            continue
    
    print(f"\\n🎉 정규화 완료! 총 {len(clean_data)}개 행 처리됨")
    return pd.DataFrame(clean_data)

In [28]:
# CSV 파일 읽기
print("기존 CSV 파일 읽는 중...")
df_original = pd.read_csv('gov24_services.csv')

print(f"원본 데이터: {len(df_original)}개 행")
print("\\n📋 처리할 샘플 데이터:")
sample_data = df_original[['title', 'cond_age', 'cond_income', 'cond_target']].head(3)
for idx, row in sample_data.iterrows():
    print(f"\\n{idx+1}. {row['title']}")
    print(f"   나이: {row['cond_age']}")
    print(f"   소득: {row['cond_income']}")
    print(f"   대상: {row['cond_target']}")

# 소량 테스트 실행 (3개만)
print("\\n=== 🧪 소량 테스트 시작 ===")
print("완전히 새로운 구조의 깔끔한 CSV를 생성합니다...")

df_clean = create_clean_normalized_csv(df_original.head(3))

# 새로운 깔끔한 CSV 저장
clean_filename = 'gov24_services_clean_normalized.csv'
df_clean.to_csv(clean_filename, index=False, encoding='utf-8-sig')

print(f"\\n✅ 깔끔한 CSV 저장: {clean_filename}")

# 결과 확인
print("\\n📊 새로운 CSV 구조:")
print(f"총 컬럼 수: {len(df_clean.columns)}개")
print("\\n새로운 컬럼들:")
for col in df_clean.columns:
    print(f"  - {col}")

# 샘플 데이터 확인
print("\\n�� 정규화 결과 샘플:")
print(df_clean[['title', 'age_min', 'age_max', 'income_type', 'income_percentage', 'target_age_group']].head())

기존 CSV 파일 읽는 중...
원본 데이터: 329개 행
\n📋 처리할 샘플 데이터:
\n1. 유아학비 (누리과정) 지원
   나이: 3세
4세
5세
   소득: nan
   대상: ○ 지원대상 : 국공립 및 사립유치원에 다니는 3~5세 유아 - '22년 1~2월생으로 유치원 입학을 희망하여 3세반에 취원한 유아도 지원 대상 - 취학대상 아동('18.1.1~12.31.출생)이 취학을 유예하는 경우, 유예한 1년에 한하여 5세 유아 무상교육비 지원(취학유예 통지서 제출) ※ 단, 지원기간은 3년을 초과할 수 없음. ○ 추가지원 : 저소득층 유아(유아학비 지원 대상 자격이 있고, 사립유치원에 다니는 법정저소득층(기초생활수급자, 차상위계층, 한부모 가정) 유아) ○ 아래의 경우 지원대상에서 제외 - 대한민국 국적을 가지지 않은 유아(난민 및 「재한외국인 처우 기본법」에 따라 법무부장관이 인정한 '특별기여자 등'은 예외적으로 인정) - 가정 양육수당 및 어린이집 보육료를 지원 받고 있는 유아 - 유치원 이용시간에 아이돌봄서비스 등과 중복지원 불가 - 해외 체류 기간이 31일째 되는 날 유아학비 지원자격 중지 ○ 자격 중지 후 유아학비를 다시 지원받기 위해서는 재신청 필요, 신청 누락으로 발생되는 지원금은 소급지원 되지 않음.
\n2. 근로·자녀장려금
   나이: 18세
70세 이상
   소득: 급여액 등(근로소득, 사업소득 또는 종교인소득의 합계)에 따라 - 근로장려금은 ㆍ 단독가구 최대 165만 원 
소득, 사업소득 또는 종교인소득이 있는 가구 - 소득요건 : 전년도 부부합산 연간 총소득이 가구 유형에 따라 정한 총 소득기준금액 미만일 것 (근로장려금) ㆍ 단독가구 : 2,200만 원 미만
소득금액 100만원 이하
재산요건 ㆍ 전년도 6월 1일 현재, 가구원 전체의 재산 합계액이 2억4천만 원 미만일 것 - 가구요건 ㆍ 단독가구 : 배우자1), 부양자녀2), 70세 이상 직계존속3)이 모두 없는 가구 ㆍ 홑벌이가구 : 배우자 또는 부양자녀 또는 70세 이상 직계존

데이터 정규화: 100%|██████████| 3/3 [00:53<00:00, 17.80s/it]

\n🎉 정규화 완료! 총 3개 행 처리됨
\n✅ 깔끔한 CSV 저장: gov24_services_clean_normalized.csv
\n📊 새로운 CSV 구조:
총 컬럼 수: 47개
\n새로운 컬럼들:
  - service_id
  - title
  - purpose_summary
  - category
  - org_name
  - org_type
  - applicant_type
  - support_type
  - apply_method
  - apply_period
  - min_amount
  - max_amount
  - phone
  - detail_url
  - age_min
  - age_max
  - age_unit
  - age_note
  - age_restriction
  - income_type
  - income_percentage
  - income_amount
  - income_unit
  - income_note
  - income_restriction
  - target_age_group
  - target_employment
  - target_education
  - target_household
  - target_special
  - target_occupation
  - target_specific_conditions
  - target_note
  - selection_method
  - required_documents
  - evaluation_criteria
  - selection_specific_conditions
  - selection_note
  - residence_type
  - specific_regions
  - residence_duration
  - residence_duration_unit
  - residence_specific_conditions
  - residence_note
  - support_summary
  - support_amount_type
  - support_no

In [29]:
def process_full_dataset_clean():
    """전체 데이터셋을 깔끔한 CSV로 변환"""
    
    print("🚀 전체 데이터셋 처리 시작...")
    print(f"총 {len(df_original)}개 행을 처리합니다.")
    
    # 전체 데이터 처리
    df_full_clean = create_clean_normalized_csv(df_original)
    
    # 최종 저장
    final_filename = 'gov24_services_final_clean2.csv'
    df_full_clean.to_csv(final_filename, index=False, encoding='utf-8-sig')
    
    print(f"\\n🎉 최종 깔끔한 CSV 저장 완료: {final_filename}")
    print(f"총 {len(df_full_clean)}개 행, {len(df_full_clean.columns)}개 컬럼")
    
    # 결과 요약
    print("\\n📊 정규화 결과 요약:")
    print(f"- 나이 조건이 있는 정책: {df_full_clean['age_min'].notna().sum()}개")
    print(f"- 소득 조건이 있는 정책: {df_full_clean['income_percentage'].notna().sum()}개")
    print(f"- 지원대상이 있는 정책: {df_full_clean['target_age_group'].notna().sum()}개")
    
    return df_full_clean

# 함수 실행
print("=== 🚀 전체 데이터 처리 시작 ===")
try:
    result = process_full_dataset_clean()
    print("\\n✅ 전체 처리 완료!")
    print(f"결과 데이터 크기: {len(result)} 행 x {len(result.columns)} 컬럼")
    
    # 샘플 데이터 확인
    print("\\n 정규화 결과 샘플:")
    sample_cols = ['title', 'age_min', 'age_max', 'income_type', 'income_percentage', 'target_age_group']
    print(result[sample_cols].head())
    
except Exception as e:
    print(f"\\n❌ 오류 발생: {e}")
    import traceback
    traceback.print_exc()

=== 🚀 전체 데이터 처리 시작 ===
🚀 전체 데이터셋 처리 시작...
총 329개 행을 처리합니다.
🧹 깔끔한 CSV 생성 시작: 329개 행
원본 서술형 데이터를 구조화된 형태로 변환합니다...


데이터 정규화:   3%|▎         | 10/329 [02:21<1:09:34, 13.09s/it]

✅ 10/329 완료


데이터 정규화:   5%|▍         | 16/329 [03:32<1:01:52, 11.86s/it]

API 호출 오류 (support): Extra data: line 20 column 1 (char 599)


...

✅ 320/329 완료


데이터 정규화:  98%|█████████▊| 321/329 [1:10:48<01:52, 14.09s/it]

API 호출 오류 (age): Extra data: line 7 column 1 (char 91)


데이터 정규화: 100%|██████████| 329/329 [1:12:19<00:00, 13.19s/it]

\n🎉 정규화 완료! 총 329개 행 처리됨
\n🎉 최종 깔끔한 CSV 저장 완료: gov24_services_final_clean2.csv
총 329개 행, 47개 컬럼
\n📊 정규화 결과 요약:
- 나이 조건이 있는 정책: 70개
- 소득 조건이 있는 정책: 27개
- 지원대상이 있는 정책: 86개
\n✅ 전체 처리 완료!
결과 데이터 크기: 329 행 x 47 컬럼
\n 정규화 결과 샘플:
                 title  age_min  age_max income_type  income_percentage  \
0       유아학비 (누리과정) 지원      3.0      5.0        None                NaN   
1             근로·자녀장려금     18.0      NaN          기타                NaN   
2       주택금융공사 월세 자금보증     35.0     35.0        None                NaN   
3       친환경 에너지절감장비 보급      NaN      NaN        None                NaN   
4  해양사고 국선 심판변론인 선정 지원     70.0      NaN        중위소득               60.0   

  target_age_group  
0             None  
1             None  
2             None  
3             None  
4             None  
